In [1]:
use "../Relative Supply/aftercollapse_merged" // After collapse_merged file is created in Ultimate Relative Supply
sum




    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
        year |     26,264    1991.762     16.4486       1964       2020
      female |     26,264    .4982866    .5000066          0          1
  experience |     26,264    22.78552     13.7232          0         58
   schooling |     26,264    2.877056    1.417441          1          5
   real_wage |     26,264     717.161    357.1733   115.8597   5898.612
-------------+---------------------------------------------------------
 weekly_wage |     26,264    593.1871     485.493    28.2335   7769.038
       q_obs |     26,264    167.0871    144.5288          1       2021
    q_weight |     26,264    265790.1    222565.3     726.23    2696058
  q_lsweight |     26,264    1.20e+07     9107980   33217.04   8.82e+07
q_lshrswei~t |     26,264    4.84e+08    3.54e+08    1109666   2.28e+09
-------------+-----------------------------------------------

In [2]:
keep n_labor_supply_weight experience schooling year female


In [3]:
gen experience_cat=.
replace experience_cat=1 if experience>=0 & experience<=9
replace experience_cat=2 if experience>=10 & experience<=19
replace experience_cat=3 if experience>=20 & experience<=29
replace experience_cat=4 if experience>=30 & experience<=39

tab experience_cat, summ(experience_cat)


(26,264 missing values generated)

(5,691 real changes made)

(5,696 real changes made)

(5,700 real changes made)

(5,693 real changes made)


experience_ |      Summary of experience_cat
        cat |        Mean   Std. Dev.       Freq.
------------+------------------------------------
          1 |           1           0       5,691
          2 |           2           0       5,696
          3 |           3           0       5,700
          4 |           4           0       5,693
------------+------------------------------------
      Total |   2.5002195   1.1178229      22,780


In [4]:
egen labor_supply_weight=sum(n_labor_supply_weight), by(experience_cat schooling female year) // labor supply weight by ....
quietly bysort year female schooling experience_cat: keep if _n==1
drop n_labor_supply_weight experience
save before_merge, replace // save before merging with predicted log wages file





file before_merge.dta saved


In [5]:
use predicted_wages // predicted log wages file is created in Predicted Wages
sort year schooling experience_cat female
merge m:1 year schooling experience_cat female using "../Relative Wage/before_merge", keep(match) nogenerate
save aftercollapse_merged, replace
sum





    Result                           # of obs.
    -----------------------------------------
    not matched                             0
    matched                             2,280  
    -----------------------------------------

file aftercollapse_merged.dta saved


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
        year |      2,280        1992    16.45556       1964       2020
      female |      2,280          .5    .5001097          0          1
experience~t |      2,280         2.5    1.118279          1          4
predicted_~s |      2,280    6.446095     .420314   5.521605   7.460212
   schooling |      2,280           3    1.414524          1          5
-------------+---------------------------------------------------------
        lswt |      2,280    1.29e+08    8.39e+07    2177664   3.63e+08


In [ ]:
egen temp1=sum(labor_supply_weight),by(year)
gen normalized_labor_supply_weight = labor_supply_weight/temp1
egen average_labor_weight=mean(normalized_labor_supply_weight),by(schooling female experience_cat) // average labor supply
drop temp1

In [7]:
* Overall high school
 egen m1=sum(predicted_log_wages*average_labor_weight*(schooling==2)), by(year)
 egen m2=sum(average_labor_weight*(schooling==2)), by(year)
 gen hs_wage=m1/m2
 drop m1 m2

In [8]:
* Overall college-plus
 egen m1=sum(predicted_log_wages*average_labor_weight*(schooling==4 | schooling==5)), by(year)
 egen m2=sum(average_labor_weight*(schooling==4 | schooling == 5)), by(year)
 gen college_wage=m1/m2
 drop m1 m2

In [9]:
gen college_hs_gap = colle_wage - hs_wage

In [10]:
table year, c(mean college_hs_gap)




--------------------------
survey    |
year      | mean(colleg~p)
----------+---------------
     1964 |       .3821373
     1965 |       .3981619
     1966 |       .4123693
     1967 |       .4232306
     1968 |       .4420724
     1969 |       .4467039
     1970 |       .4374728
     1971 |       .4297104
     1972 |       .4567652
     1973 |       .4394598
     1974 |       .4370584
     1975 |       .4203458
     1976 |       .4123511
     1977 |       .4103203
     1978 |       .3933167
     1979 |       .3803658
     1980 |       .3879089
     1981 |       .3854618
     1982 |       .3847647
     1983 |       .4120359
     1984 |       .4410009
     1985 |       .4543099
     1986 |       .4708719
     1987 |       .4844832
     1988 |       .4877148
     1989 |       .4988132
     1990 |       .5269022
     1991 |        .521677
     1992 |       .5474963
     1993 |       .5497427
     1994 |       .5773473
     1995 |       .5788021
     1996 |       .5807238
     1997 |  

In [11]:
collapse college_hs_gap, by(year)
save "../Data/relative_wage", replace



file ../Data/relative_wage.dta saved
